In [ ]:
## magic commands
%reload_ext autoreload
%autoreload 2
%matplotlib inline
## import required packages
from fastai.text import *
from fastai.imports import *
import pandas as pd
import re

In [ ]:
## Colab google drive stuff
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'my_model/'

Mounted at /content/gdrive


In [ ]:
!pip install --no-cache-dir spacy==2.3.5

In [ ]:
## prepare data from chat.txt
path = Path(base_dir)
final_data=[]
data=[]
with open(path/'chat.txt') as f:
  line=''
  for char in f.read():
    line+=char
    if char=='\n':
      try:
        timestamp,second_part=line.split('-')
        sender,msg=second_part.split(':')
        final_data.append([timestamp.strip(),sender.strip(),msg.strip()])
      except:
        pass
      data.append(line)
      line=''
      continue

In [ ]:
#prepare dataframe from chat.txt
df = pd.DataFrame(final_data, columns=['timestamp', 'sender', 'text'])

In [ ]:
#import model requriements
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing
from tensorflow.keras import preprocessing, utils
import os
import yaml

In [ ]:
#split data from dataframe into questions and answers list
questions=list()
answers=list()

for index, row in df.iterrows():
  if row['sender']=='Meet Rajput':
    answers.append(row['text'])
  else:
    questions.append(row['text'])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( ' ' + answers_with_tags[i] + ' ' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 3938


In [ ]:
from gensim.models import Word2Vec
import re

In [ ]:
#tokenize the dataframe
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary

In [ ]:
#equalize both questions and answers
answers=answers[:2991]

In [ ]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

(2991, 39) 39


In [ ]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(2991, 14) 14


In [ ]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(2991, 14, 3938)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 39)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 14)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 39, 200)      787600      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 14, 200)      787600      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 )
model.save( 'model.h5' )

Epoch 1/150
60/60 [==============================] - 36s 409ms/step - loss: 6.5486
Epoch 2/150
60/60 [==============================] - 24s 404ms/step - loss: 4.7160
Epoch 3/150
60/60 [==============================] - 24s 390ms/step - loss: 4.3372
Epoch 4/150
60/60 [==============================] - 25s 417ms/step - loss: 4.1593
Epoch 5/150
60/60 [==============================] - 23s 384ms/step - loss: 4.0584
Epoch 6/150
60/60 [==============================] - 23s 380ms/step - loss: 4.0054
Epoch 7/150
60/60 [==============================] - 24s 394ms/step - loss: 3.9412
Epoch 8/150
60/60 [==============================] - 22s 369ms/step - loss: 3.9025
Epoch 9/150
60/60 [==============================] - 25s 416ms/step - loss: 3.8596
Epoch 10/150
60/60 [==============================] - 22s 372ms/step - loss: 3.8272
Epoch 11/150
60/60 [==============================] - 25s 415ms/step - loss: 3.7975
Epoch 12/150
60/60 [==============================] - 22s 374ms/step - loss: 3.7700
E

In [ ]:
def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)

    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()

    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]

    print( decoded_translation )

Enter question : Tera set konsa h
1/1 [==============================] - 0s 29ms/step
 test ka h karne me karne ki karne ki karne ki karne ki karne ki
Enter question : Aaj kya hoga
1/1 [==============================] - 0s 30ms/step
 ho karne me karne ki karne ki karne ki karne ki karne ki karne ki
Enter question : One piece AOT naruto kede lge the bhot
1/1 [==============================] - 0s 30ms/step
 karne karne karne karne karne karne karne karne karne karne karne karne karne karne karne


KeyboardInterrupt: Interrupted by user